### Курсовой проект для курса "Python для Data Science"

Материалы к проекту (файлы):
train.csv
test.csv

Задание:
Используя данные из обучающего датасета (train.csv), построить модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели, предсказать цены для квартир из тестового датасета (test.csv).

Целевая переменная:
Price

Метрика качества:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Требования к решению:
1. R2 > 0.6
2. Тетрадка Jupyter Notebook с кодом Вашего решения, названная по образцу {ФИО}_solution.ipynb, пример SShirkin_solution.ipynb
3. Файл CSV с прогнозами целевой переменной для тестового датасета, названный по образцу {ФИО}_predictions.csv, пример SShirkin_predictions.csv 
Файл должен содержать два поля: Id, Price и в файле должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки сдачи:
Cдать проект нужно в течение 72 часов после окончания последнего вебинара. Оценки работ, сданных до дедлайна, будут представлены в виде рейтинга, ранжированного по заданной метрике качества. Проекты, сданные после дедлайна или сданные повторно, не попадают в рейтинг, но можно будет узнать результат.

Рекомендации для файла с кодом (ipynb):
1. Файл должен содержать заголовки и комментарии (markdown)
2. Повторяющиеся операции лучше оформлять в виде функций
3. Не делать вывод большого количества строк таблиц (5-10 достаточно)
4. По возможности добавлять графики, описывающие данные (около 3-5)
5. Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
6. Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
7. Весь проект должен быть в одном скрипте (файл ipynb).
8. Допускается применение библиотек Python и моделей машинного обучения,
которые были в данном курсе.

Описание датасета:
- Id - идентификационный номер квартиры
- DistrictId - идентификационный номер района
- Rooms - количество комнат
- Square - площадь
- LifeSquare - жилая площадь
- KitchenSquare - площадь кухни
- Floor - этаж
- HouseFloor - количество этажей в доме
- HouseYear - год постройки дома
- Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
- Social_1, Social_2, Social_3 - социальные показатели местности
- Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
- Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
- Price - цена квартиры


### Опробованные варианты:
#### 1) Базовая модель: только исключены показатели с пропусками
RandomForestRegressor(max_depth=10, n_estimators=100, random_state=123)  
Локальный результат: r2 = 0.7345183880191818  
Kaggle: r2 = 0.72001

#### 2) Подбор гиперпараметров
'max_depth': 16,  
'n_estimators': 175

#### 3) Работа с признаками
 - DistrictId заменен медианными значениями цены (Price_median) по каждому району  


In [54]:
import pandas as pd
import numpy as np

In [55]:
import matplotlib
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [56]:
import warnings
warnings.filterwarnings('ignore')

In [57]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [58]:
houses_train = pd.read_csv('./train.csv')

In [59]:
houses_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


### Обработка признаков. 
Проверим пропуски, уникальные значения и основные показатели

In [60]:
houses_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [61]:
houses_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,10000.0,8383.407700,4859.019020,0.000000,4169.500000,8394.500000,12592.500000,1.679800e+04
DistrictId,10000.0,50.400800,43.587592,0.000000,20.000000,36.000000,75.000000,2.090000e+02
Rooms,10000.0,1.890500,0.839512,0.000000,1.000000,2.000000,2.000000,1.900000e+01
Square,10000.0,56.315775,21.058732,1.136859,41.774881,52.513310,65.900625,6.410652e+02
LifeSquare,7887.0,37.199645,86.241209,0.370619,22.769832,32.781260,45.128803,7.480592e+03
KitchenSquare,10000.0,6.273300,28.560917,0.000000,1.000000,6.000000,9.000000,2.014000e+03
Floor,10000.0,8.526700,5.241148,1.000000,4.000000,7.000000,12.000000,4.200000e+01
HouseFloor,10000.0,12.609400,6.775974,0.000000,9.000000,13.000000,17.000000,1.170000e+02
HouseYear,10000.0,3990.166300,200500.261427,1910.000000,1974.000000,1977.000000,2001.000000,2.005201e+07
Ecology_1,10000.0,0.118858,0.119025,0.000000,0.017647,0.075424,0.195781,5.218671e-01


In [62]:
houses_train.nunique()

Id               10000
DistrictId         205
Rooms                9
Square           10000
LifeSquare        7887
KitchenSquare       58
Floor               33
HouseFloor          44
HouseYear           97
Ecology_1          129
Ecology_2            2
Ecology_3            2
Social_1            51
Social_2           142
Social_3            30
Healthcare_1        79
Helthcare_2          7
Shops_1             16
Shops_2              2
Price            10000
dtype: int64

Преобразуем категориальный признак Ecology_2, Ecology_3, Shops_2. В начале рассматривались значения этих признаков, все они имеют начения A и B, заменяем на 0 и 1

In [63]:
houses_train.replace({'Ecology_2': {'A': 0, 'B': 1}, 'Ecology_3': {'A': 0, 'B': 1}, 'Shops_2': {'A': 0, 'B': 1}}, inplace=True)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(houses_train.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price'], axis='columns'), houses_train['Price'], test_size=0.25, random_state=123)

Построение и обучение модели

In [65]:
model_rfr = RandomForestRegressor(max_depth=16, n_estimators=175, random_state=123)
model_rfr.fit(X_train, y_train)
predict_1 = model_rfr.predict(X_test)
r2(y_test, predict_1)

0.7403388280755667

### Загрузка тестовой выборки

In [66]:
houses_test = pd.read_csv('./test.csv')
houses_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [67]:
houses_test.nunique()

Id               5000
DistrictId        201
Rooms               8
Square           5000
LifeSquare       3959
KitchenSquare      38
Floor              35
HouseFloor         41
HouseYear          97
Ecology_1         130
Ecology_2           2
Ecology_3           2
Social_1           51
Social_2          143
Social_3           30
Healthcare_1       79
Helthcare_2         7
Shops_1            16
Shops_2             2
dtype: int64

In [68]:
houses_test.replace({'Ecology_2': {'A': 0, 'B': 1}, 'Ecology_3': {'A': 0, 'B': 1}, 'Shops_2': {'A': 0, 'B': 1}}, inplace=True)

Предсказание на базовой модели (исключены признаки с пропусками) и на подобранных гиперпараметрах:  
'max_depth': 16,  
'n_estimators': 175

In [69]:
test_predict_2 = model_rfr.predict(houses_test.set_index('Id').drop(['LifeSquare', 'Healthcare_1'], axis='columns'))

In [70]:
houses_test['Price'] = test_predict_2

In [71]:
houses_test[['Id', 'Price']].to_csv('kaggle_20200825_1541.csv', index=False)

Проверим степень влияния каждого признака

In [72]:
f_i = pd.DataFrame()
f_i['Name'] = houses_test.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price'], axis='columns').columns.tolist()
f_i['Values'] = model_rfr.feature_importances_
f_i.sort_values('Values', ascending=False).reset_index(drop=True)

,Name,Values
0,Square,0.403806
1,Social_2,0.111587
2,Social_1,0.091726
3,Rooms,0.079661
4,Social_3,0.061946
5,DistrictId,0.049077
6,Ecology_1,0.046839
7,HouseYear,0.039587
8,Floor,0.031603
9,KitchenSquare,0.029236


Заменим DistrictId медианными значениями цены по каждому району, назовем его Price_median

In [73]:
median_price_by_dist_id = houses_train.groupby('DistrictId')[['Price']].median().reset_index()
median_price_by_dist_id

,DistrictId,Price
0,0,165963.054142
1,1,183663.443595
2,2,208539.501373
3,3,169094.013281
4,4,278639.482329
...,...,...
200,202,394150.861857
201,205,220501.566180
202,207,426186.409334
203,208,431137.654083


In [74]:
houses_train = houses_train.merge(median_price_by_dist_id, on='DistrictId', how='left', suffixes=('', '_median'))

In [80]:
houses_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Price_median
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,...,1,33,7976,5,NaN,0,11,1,184966.930730,203602.408898
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,...,1,46,10309,1,240.0,1,16,1,300009.450063,210694.850106
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,...,1,34,7759,0,229.0,1,3,1,220925.908524,245978.794474
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,...,1,23,5735,3,1084.0,0,5,1,175616.227217,151557.904767
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,...,1,35,5776,1,2078.0,2,4,1,150226.531644,178829.166450


Снова разобъем выборку на train и test, удалив из нее 'LifeSquare', 'Healthcare_1', 'Price', 'DistrictId'

In [76]:
X_train, X_test, y_train, y_test = train_test_split(
    houses_train.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price', 'DistrictId'], axis='columns'), 
    houses_train['Price'], 
    test_size=0.25,
    random_state=123)

Снова обучаем

In [77]:
model_rfr = RandomForestRegressor(max_depth=16, n_estimators=175, random_state=123)
model_rfr.fit(X_train, y_train)
predict_1 = model_rfr.predict(X_test)
r2(y_test, predict_1)

0.7432353600717603

In [79]:
f_i = pd.DataFrame()
f_i['Name'] = houses_train.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price', 'DistrictId'], axis='columns').columns.tolist()
f_i['Values'] = model_rfr.feature_importances_
f_i.sort_values('Values', ascending=False).reset_index(drop=True)

,Name,Values
0,Square,0.423963
1,Price_median,0.313253
2,HouseYear,0.034819
3,Rooms,0.031997
4,Social_3,0.029595
5,Floor,0.027688
6,KitchenSquare,0.025601
7,Social_2,0.024854
8,HouseFloor,0.023879
9,Social_1,0.022911


Загружаем тестовую выборку

In [122]:
houses_test = pd.read_csv('./test.csv')
houses_test.replace({'Ecology_2': {'A': 0, 'B': 1}, 'Ecology_3': {'A': 0, 'B': 1}, 'Shops_2': {'A': 0, 'B': 1}}, inplace=True)
houses_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,1,1,11,2748,1,NaN,0,0,1
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,1,1,6,1437,3,NaN,0,2,1
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,1,1,30,7538,87,4702.0,5,5,1
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,1,1,23,4583,3,NaN,3,3,1
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,1,1,2,629,1,NaN,0,0,0


In [123]:
houses_test = houses_test.merge(median_price_by_dist_id, on='DistrictId', how='left')

In [125]:
houses_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,1,1,11,2748,1,NaN,0,0,1,151557.904767
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,1,1,6,1437,3,NaN,0,2,1,195610.960042
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,1,1,30,7538,87,4702.0,5,5,1,526438.458919
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,1,1,23,4583,3,NaN,3,3,1,196429.659238
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,1,1,2,629,1,NaN,0,0,0,146171.433190


In [127]:
houses_test = houses_test.rename(columns={"Price": "Price_median"})

Заполняем недостающие значения Price медианами

In [129]:
houses_test['Price_median'] = houses_test['Price_median'].fillna(houses_test['Price_median'].median())

In [130]:
houses_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     3959 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   int64  
 11  Ecology_3      5000 non-null   int64  
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   2623 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

Предсказание на следующей модели:  
 - исключены признаки с пропусками
 - DistrictId заменен медианными значениями цены (Price_median) по каждому району  

Подобранные гиперпараметры:  
'max_depth': 16,  
'n_estimators': 175

In [131]:
test_predict_3 = model_rfr.predict(houses_test.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'DistrictId'], axis='columns')) 

In [132]:
houses_test['Price'] = test_predict_3

In [133]:
houses_test[['Id', 'Price']].to_csv('kaggle_20200825_1843.csv', index=False)